In [1]:
ans_file=""
text=""
question=""

In [2]:
# Parameters
text = "Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has\r\nbeen the industry's standard dummy text ever since the 1500s, when an unknown printer took\r\na galley of type and scrambled it to make a type specimen book. It has survived not only five\r\ncenturies, but also the leap into electronic typesetting, remaining essentially unchanged. It was\r\npopularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages,\r\nand more recently with desktop publishing software like Aldus PageMaker including versions of\r\nLorem Ipsum. Contrary to popular belief, Lorem Ipsum is not simply random text. It has roots in\r\na piece of classical Latin literature from 45 BC, making it over 2000 years old. Richard\r\nMcClintock, a Latin professor at Hampden-Sydney College in Virginia, looked up one of the\r\nmore obscure Latin words, consectetur, from a Lorem Ipsum passage, and going through the\r\ncites of the word in classical literature, discovered the undoubtable source. Lorem Ipsum comes\r\nfrom sections 1.10.32 and 1.10.33 of \"de Finibus Bonorum et Malorum\" (The Extremes of Good\r\nand Evil) by Cicero, written in 45 BC. This book is a treatise on the theory of ethics, very popular\r\nduring the Renaissance. The first line of Lorem Ipsum, \"Lorem ipsum dolor sit amet..\", comes\r\nfrom a line in section 1.10.32.\r\n\r\nThe standard chunk of Lorem Ipsum used since the 1500s is reproduced below for those\r\ninterested. Sections 1.10.32 and 1.10.33 from \"de Finibus Bonorum et Malorum\" by Cicero are\r\nalso reproduced in their exact original form, accompanied by English versions from the 1914\r\ntranslation by H. Rackham.\r\n\r\n\r\n\r\nIt is a long established fact that a reader will be distracted by the readable content of a page\r\nwhen looking at its layout. The point of using Lorem Ipsum is that it has a more-or-less normal\r\ndistribution of letters, as opposed to using 'Content here, content here', making it look like\r\nreadable English. Many desktop publishing packages and web page editors now use Lorem\r\nIpsum as their default model text, and a search for 'lorem ipsum' will uncover many web sites\r\nstill in their infancy. Various versions have evolved over the years, sometimes by accident,\r\nsometimes on purpose (injected humour and the like).\r\n\r\n\r\n\r\nThere are many variations of passages of Lorem Ipsum available, but the majority have suffered\r\nalteration in some form, by injected humour, or randomised words which don't look even\r\nslightly believable. If you are going to use a passage of Lorem Ipsum, you need to be sure there\r\nisn't anything embarrassing hidden in the middle of text. All the Lorem Ipsum generators on the\r\nInternet tend to repeat predefined chunks as necessary, making this the first true generator on\r\nthe Internet. It uses a dictionary of over 200 Latin words, combined with a handful of model\r\nsentence structures, to generate Lorem Ipsum which looks reasonable. The generated Lorem\r\nIpsum is therefore always free from repetition, injected humour, or non-characteristic words\r\netc.\r\n\x0c"
question = "What is Lorem Ipsum ?"
ans_file = "QAS_ans.txt"


In [3]:
#Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import re
import time
import papermill as pm
import warnings
warnings.filterwarnings("ignore")

In [4]:
#Loading Model and Encoder
model=torch.load("MODEL_APR_23")
model.eval()
tokenizer = torch.load("tokenizer_encoder.unknown")

In [5]:
def question_answer_A(question, text):
    
    #tokenize question and text in ids as a pair
    input_ids = tokenizer.encode(question, text)
    
    #string version of tokenized ids
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    
    #segment IDs
    #first occurence of [SEP] token
    sep_idx = input_ids.index(tokenizer.sep_token_id)

    #number of tokens in segment A - question
    num_seg_a = sep_idx+1

    #number of tokens in segment B - text
    num_seg_b = len(input_ids) - num_seg_a
    
    #list of 0s and 1s
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    
    assert len(segment_ids) == len(input_ids)
    
    #model output using input_ids and segment_ids
    output = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))
    #reconstructing the answer
    answer_start = torch.argmax(output.start_logits)
    answer_end = torch.argmax(output.end_logits)
    answer = 'unable to find answer'
    if answer_end >= answer_start:
        answer = tokens[answer_start]
        for i in range(answer_start+1, answer_end+1):
            if tokens[i][0:2] == "##":
                answer += tokens[i][2:]
            else:
                answer += " " + tokens[i]
                
    if answer.startswith("[CLS]"):
        answer = "Unable to find the answer to your question."
    
    return torch.max(output.start_logits), answer.capitalize() #score ans

In [6]:
def chunk_text(text, max_len):
    # split the text into sentences
    sentences = re.findall('[^.!?]+[.!?]', text)
    
    # initialize variables
    current_len = 0
    current_chunk = []
    chunks = []
    
    # iterate over each sentence
    for sentence in sentences:
        # add the sentence to the current chunk
        current_chunk.append(sentence)
        
        # update the length of the current chunk
        current_len += len(sentence)
        
        # if the length of the current chunk is greater than or equal to the max length
        if current_len >= max_len:
            # add the current chunk to the list of chunks
            chunks.append(' '.join(current_chunk))
            
            # reset the current chunk and length
            current_chunk = []
            current_len = 0
    
    # add any remaining sentences as a final chunk
    if current_chunk:
        chunks.append(' '.join(current_chunk))
    
    return chunks

In [7]:
def get_better_ans(question, text):
  chunk_of_text=chunk_text(text,450)
  scores = {}
  for i in chunk_of_text:
    scores.update({question_answer_A(question, i)[0] : question_answer_A(question, i)[1] })
  v = list(scores.values())
  k = list(scores.keys())
  return (v[k.index(max(k))])

In [8]:
print(text)
print(question)
x = get_better_ans(question, text)


Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has
been the industry's standard dummy text ever since the 1500s, when an unknown printer took
a galley of type and scrambled it to make a type specimen book. It has survived not only five
centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was
popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages,
and more recently with desktop publishing software like Aldus PageMaker including versions of
Lorem Ipsum. Contrary to popular belief, Lorem Ipsum is not simply random text. It has roots in
a piece of classical Latin literature from 45 BC, making it over 2000 years old. Richard
McClintock, a Latin professor at Hampden-Sydney College in Virginia, looked up one of the
more obscure Latin words, consectetur, from a Lorem Ipsum passage, and going through the
cites of the word in classical literature, discovered the undoubtable sour

In [9]:
print(x)
f = open(ans_file, "w")
f.write(x)
f.close()

Dummy text of the printing and typesetting industry
